In [52]:
import pandas as pd
import numpy as np

In [200]:
bipartite = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\all_candidates_state_bipartite_weighted_network.csv")
federal_top100 = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\federal_contributor_top100_contributors_network.csv")
state_top100 = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\state_contributor_top100_contributors_network.csv")
test = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\test_data.csv")
training = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\training_data.csv")
winning_candidates = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\winning_candidates_state_bipartite_weighted_network.csv")

C:\Users\Alex Novak\AppData\Local\Temp\ipykernel_30060\648240549.py:5: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  training = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\training_data.csv")


In [114]:
#Incumbency is a big advantage
#If dems and reps are split they could just be bribing everyone
#If its individual and non-governor could use zip code and assume they are donating to someone in that area

In [201]:
for i in (31,38,40):
    col = training.columns[i]
    training[col] = [float(j) if j != "#NAME?" else np.nan for j in training[col]]

In [194]:
training = training[['state', "zip_code",
       'contributor_type', 'winner_ratio', 'candidacy_count',
       'candidacy_democratic_count', 'candidacy_republican_count',
       'contribution_count', 'contribution_democratic_count',
       'contribution_republican_count', 'politician_challenger_count',
       'politician_count', 'politician_democratic_count',
       'politician_incumbency_count', 'politician_open_pos_count',
       'politician_republican_count', 'contribution_democratic_sum_2010_usd',
       'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd',
       'governor_contributions_sum_2010_usd',
       'house_and_assembly_contributions_sum_2010_usd',
       'senate_contributions_sum_2010_usd',
       'us_house_contributions_sum_2010_usd',
       'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio',
       'candidacy_republican_ratio', 'contribution_democratic_count_ratio',
       'contribution_republican_count_ratio', 'governor_contribution_ratio',
       'house_and_assembly_contribution_ratio', 'politician_challenger_ratio',
       'politician_democratic_ratio', 'politician_incumbency_ratio',
       'politician_open_pos_ratio', 'politician_republican_ratio',
       'senate_contribution_ratio', 'us_house_contribution_ratio',
       'us_senate_contribution_ratio']]

#training = training[training["state"] == "KY"]

In [202]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

training = training.dropna()

# Split features and target
X = training.drop(columns=["winner_ratio"])
y = training["winner_ratio"]

# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_cols = X.select_dtypes(include=["number"]).columns.tolist()

# Preprocessing: One-hot encode categorical features, passthrough numeric
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

# Define XGBoost regressor
xgb_regressor = xgb.XGBRegressor(
    objective="reg:squarederror",
    eval_metric="rmse",
    n_estimators=25000,
    learning_rate=0.005,
    max_depth=8,
    random_state=42,
    njobs=-1
)

# Combine preprocessing and model in a pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_regressor)
])

# Split data into train/test for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

# Fit the pipeline
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

C:\Users\Alex Novak\anaconda3\envs\Tobacco_research\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:38:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "njobs" } are not used.

  warnings.warn(smsg, UserWarning)


In [158]:
#X_test["prediction"] = y_pred
#X_test["true"] = y_test
#X_test.head(20)

In [196]:
# Predict and evaluate
#y_pred = y_pred[X_test["state"] == "KY"]
#y_test = y_test[X_test["state"] == "KY"]
y_pred = np.clip(y_pred, 0, 1) #Enforce it being between 0 and 1
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse:.4f}")

RMSE: 0.2716


C:\Users\Alex Novak\anaconda3\envs\Tobacco_research\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [162]:
X_test["error"] = X_test["prediction"] - X_test["true"]
X_test[(X_test["error"] > 0.25) | (X_test["error"] < -0.25)]

,state,contributor_type,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,prediction,true,error
22565,KY,Individual,2,0,2,3,0,3,0,2,...,0.000000,0.500000,0.500000,1.000000,0.000000,0.0,0.0,0.452925,0.0,0.452925
172808,KY,Individual,8,4,4,8,4,4,2,8,...,0.500000,0.000000,0.750000,0.500000,0.327943,0.0,0.0,0.525826,0.0,0.525826
116065,KY,Individual,4,4,0,6,6,0,0,3,...,1.000000,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.752114,0.5,0.252114
29575,KY,Individual,2,2,0,3,3,0,0,2,...,1.000000,0.500000,0.500000,0.000000,0.000000,0.0,0.0,0.739646,1.0,-0.260354
58973,KY,Individual,2,2,0,2,2,0,1,2,...,1.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.670546,1.0,-0.329454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27935,KY,Individual,2,2,0,2,2,0,0,2,...,1.000000,0.000000,1.000000,0.000000,0.391531,0.0,0.0,0.952315,0.0,0.952315
41381,KY,Individual,2,2,0,6,6,0,0,2,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.349204,0.0,0.349204
5269,KY,Individual,2,2,0,3,3,0,0,2,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.329965,0.0,0.329965
97698,KY,Individual,2,0,2,6,0,6,1,2,...,0.000000,0.000000,0.500000,1.000000,0.000000,0.0,0.0,0.444848,1.0,-0.555152


In [203]:
test = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\test_data.csv")

In [197]:
test = pd.read_csv(r"campaign-contributions-in-the-us-spring-2025\test_data.csv")
#test = test[test["state"] == "KY"]
test = test[['state', "zip_code",
       'contributor_type', 'winner_ratio', 'candidacy_count',
       'candidacy_democratic_count', 'candidacy_republican_count',
       'contribution_count', 'contribution_democratic_count',
       'contribution_republican_count', 'politician_challenger_count',
       'politician_count', 'politician_democratic_count',
       'politician_incumbency_count', 'politician_open_pos_count',
       'politician_republican_count', 'contribution_democratic_sum_2010_usd',
       'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd',
       'governor_contributions_sum_2010_usd',
       'house_and_assembly_contributions_sum_2010_usd',
       'senate_contributions_sum_2010_usd',
       'us_house_contributions_sum_2010_usd',
       'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio',
       'candidacy_republican_ratio', 'contribution_democratic_count_ratio',
       'contribution_republican_count_ratio', 'governor_contribution_ratio',
       'house_and_assembly_contribution_ratio', 'politician_challenger_ratio',
       'politician_democratic_ratio', 'politician_incumbency_ratio',
       'politician_open_pos_ratio', 'politician_republican_ratio',
       'senate_contribution_ratio', 'us_house_contribution_ratio',
       'us_senate_contribution_ratio']]

In [204]:
test_pred = pipeline.predict(test)
test_pred = np.clip(test_pred, 0, 1)

In [205]:
test["index"] = list(range(len(test)))
test["winner_ratio"] = test_pred
test[["index", "winner_ratio"]].to_csv("prediction.csv", index=False)

In [ ]:
#Current best: .2834
#Can maybe slightly improve by clipping to 0.05, 0.95

In [134]:
test[(test["state"] == "KY") & (test["politician_incumbency_ratio"] == 1)]

,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,...,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,index
395,KY,Individual,0.000000,2,0,2,3,0,3,0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,395
853,KY,Individual,0.467534,2,1,1,2,1,1,0,...,0.000000,0.0,0.5,1.0,0.0,0.5,0.0,0.0,0.0,853
2805,KY,Individual,0.000000,2,0,2,3,0,3,0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2805
5211,KY,Individual,0.767756,4,4,0,6,6,0,0,...,0.580169,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5211
5595,KY,Non-Individual,1.000000,2,0,2,2,0,2,0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,5595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110107,KY,Individual,0.000129,2,0,2,2,0,2,0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,110107
111998,KY,Individual,0.000000,2,0,2,2,0,2,0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,111998
113484,KY,Individual,0.353633,2,1,1,4,2,2,0,...,0.000000,0.0,0.5,1.0,0.0,0.5,0.0,0.0,0.0,113484
113573,KY,Individual,1.000000,2,2,0,2,2,0,0,...,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,113573
